# Independent Cascade Model on Facebook Social Circles Dataset

Code borrowed from https://github.com/doinab/SN-influence-maximization/

Dataset description: https://snap.stanford.edu/data/egonets-Facebook.html

This dataset consists of 'circles' (or 'friends lists') from Facebook.
- Nodes	4039
- Edges	88234
- Average clustering coefficient	0.6055
- Diameter (longest shortest path)	8

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
import networkx as nx
import random
import numpy
import math
import time
import sys

In [2]:
# Simulates the Independent Cascade propagation model 
# on graph G starting with the seed nodes in "a".
# Returns the number of nodes reached by the propagation
# -> Time complexity: a very worst case of O(V^3) (lower with lower "p")
# -> Memory complexity: O(V)
def IC_model(G, a, p):              # a: the set of initial active nodes
                                    # p: the system-wide probability of influence on an edge, in [0,1]
    A = set(a)                      # A: the set of active nodes, initially a
    B = set(a)                      # B: the set of nodes activated in the last completed iteration
    converged = False

    while not converged:
        nextB = set()
        for n in B:
            for m in set(G.neighbors(n)) - A:
                prob = random.random()	# in the range [0.0, 1.0)
                if prob <= p:
                    nextB.add(m)
        B = set(nextB)
        if not B:
            converged = True
        A |= B

    return len(A)

In [3]:
# Simulates the Weighted Cascade propagation model 
# on graph G starting with the seed nodes in "a".
# Returns the number of nodes reached by the propagation
# -> Time complexity: a very worst case of O(V^3)
# -> Memory complexity: O(V)
def WC_model(G, a):                 # a: the set of initial active nodes
                                    # each edge from node u to v is assigned probability 1/in-degree(v) of activating v
    A = set(a)                      # A: the set of active nodes, initially a
    B = set(a)                      # B: the set of nodes activated in the last completed iteration
    converged = False
 
    if nx.is_directed(G):
        my_degree_function = G.in_degree
    else:
        my_degree_function = G.degree

    while not converged:
        nextB = set()
        for n in B:
            for m in set(G.neighbors(n)) - A:
                prob = random.random()	# in the range [0.0, 1.0)
                p = 1.0/my_degree_function(m)
                if prob <= p:
                    nextB.add(m)
        B = set(nextB)
        if not B:
            converged = True
        A |= B

    return len(A)


In [4]:
# Evaluates a given seed set A,
# simulated "no_simulations" times
# Returns a tuple: the mean, stdev, and 95% confidence interval
def evaluate(G, A, p, no_simulations, model):
    results = []

    if model == 'WC':
        for i in range(no_simulations):
            results.append(WC_model(G, A))
    elif model == 'IC':
        for i in range(no_simulations):
            results.append(IC_model(G, A, p))

    return numpy.mean(results), numpy.std(results), 1.96 * numpy.std(results) / math.sqrt(no_simulations)

In [5]:
# Evaluates "no_samples" random seed sets A of size "k",
# each simulated "no_simulations" times.
# Returns a list of "no_samples" tuples (mean, stdev, and 95% confidence interval)
def RND_evaluate(G, k, p, no_samples, no_simulations, model):
    results = []

    for i in range(no_samples):
        A = random.sample(G.nodes(), k)
        results.append(evaluate(G, A, p, no_simulations, model))
    return results

In [6]:
p = 0.01                                # "p" only matters for IC_model
k = 10
model = 'IC'
no_simulations = 100
no_samples = 10

# file = 'soc-Epinions1.txt'
# file = 'wiki-Vote.txt'
# file = 'amazon0302.txt'
# file = 'web-Google.txt'
# G = nx.read_edgelist(file, comments='#', delimiter='\t', create_using=nx.DiGraph(), nodetype=int, data=False)
file = 'facebook_combined.txt'
G = nx.read_edgelist(file, comments='#', delimiter=' ', create_using=nx.Graph(), nodetype=int, data=False)

tstart = time.process_time()
print(RND_evaluate(G, k, p, no_samples, no_simulations, model))
#print(evaluate(G, input_A(), p, no_simulations, model))
print("Elapsed time: ", time.process_time() - tstart)

[(195.15, 81.35445593205083, 15.94547336268196), (65.61, 76.87143747842887, 15.066801745772059), (16.7, 8.350449089719666, 1.6366880215850543), (144.8, 81.63516399199551, 16.00049214243112), (142.41, 110.41649288036638, 21.64163260455181), (214.9, 87.81588694535859, 17.211913841290283), (168.66, 81.37004608576795, 15.948529032810518), (163.48, 63.68398228754229, 12.48206052835829), (30.19, 51.33316569236696, 10.061300475703925), (169.45, 92.51793069454159, 18.13351441613015)]
Elapsed time:  3.578125


# Influence maximization heuristics on wiki-Vote data.

Dataset description

https://snap.stanford.edu/data/wiki-Vote.html
  
Wikipedia is a free encyclopedia written collaboratively by volunteers around the world. A small part of Wikipedia contributors are administrators, who are users with access to additional technical features that aid in maintenance. In order for a user to become an administrator a Request for adminship (RfA) is issued and the Wikipedia community via a public discussion or a vote decides who to promote to adminship. Using the latest complete dump of Wikipedia page edit history (from January 3 2008) we extracted all administrator elections and vote history data. This gave us 2,794 elections with 103,663 total votes and 7,066 users participating in the elections (either casting a vote or being voted on). Out of these 1,235 elections resulted in a successful promotion, while 1,559 elections did not result in the promotion. About half of the votes in the dataset are by existing admins, while the other half comes from ordinary Wikipedia users.

The network contains all the Wikipedia voting data from the inception of Wikipedia till January 2008. Nodes in the network represent wikipedia users and a directed edge from node i to node j represents that user i voted on user j.

- Nodes	7115
- Edges	103689
- Average clustering coefficient	0.1409
- Diameter (longest shortest path)	7

In [7]:
import json
import heapq as hq

In [8]:
# [Kempe et al.] "The high-degree heuristic chooses nodes v in order of decreasing degrees. 
# 'degree centrality'
# -> Calculates the k nodes of highest degree
# -> Time complexity: O(V log (k))
# -> Memory complexity: Theta(k)
# H is the selected set of high influence nodes.
def high_degree_nodes(k, G):

    if nx.is_directed(G):
        my_degree_function = G.out_degree
    else:
        my_degree_function = G.degree
    x=list(G.nodes())
    # the top k nodes to be returned; initialization with first k elements
    H = [(my_degree_function(i), i) for i in x[0:k]] 
    hq.heapify(H)

    # iterate through the remaining nodes; add to heap if larger than heap root
    for i in x[k:]: 
        if my_degree_function(i) > H[0][0]:
            hq.heappushpop(H, (my_degree_function(i), i))
 
    return H


In [9]:
# The SingleDiscount algorithm by [Chen et al., KDD'09] for any cascade model.
# -> Calculates the k nodes of highest degree, making discounts if direct neighbours are already chosen
# -> Time complexity: O(V k^2)
# -> Memory complexity: Theta(k)
# D is the selected set of high influence nodes.
def single_discount_high_degree_nodes(k, G):

    if nx.is_directed(G):
        my_degree_function = G.out_degree
    else:
        my_degree_function = G.degree

    D = []

    for i in range(k):
        # find the node of max out_degree, discounting any out-edge
        # to a node already in D
        maxoutdeg_i = -1
        v_i = -1
        for v in list(set(G.nodes()) - set(D)):
    	    outdeg = my_degree_function(v)
    	    for u in D:
                if G.has_edge(v, u):
                    outdeg -= 1
    	    if outdeg > maxoutdeg_i:
                maxoutdeg_i = outdeg
                v_i = v

        D.append(v_i)

    return D

In [17]:
# The approximate greedy algorithm by [Kempe et al.] for any cascade model.
# -> Calculates the k nodes of supposedly max influence, and that influence
# -> Single-thread
# -> Time complexity: O(V k Time(evaluate))
# -> Memory complexity: Theta(k + Memory(evaluate))
# S is the selected set of high influence nodes.
def general_greedy(k, G, p, no_simulations, model):
    S = []

    for i in range(k):
        maxinfl_i = (-1, -1)
        v_i = -1
        for v in list(set(G.nodes()) - set(S)):
            eval_tuple = evaluate(G, S+[v], p, no_simulations, model)
            #if v%100==0:
            #    print(i,v, eval_tuple[0], eval_tuple[2])
            if eval_tuple[0] > maxinfl_i[0]:
                maxinfl_i = (eval_tuple[0], eval_tuple[2])
                v_i = v

        print('Simple Greedy Detail', i, v_i, maxinfl_i)
        S.append(v_i)

    return S, maxinfl_i

In [18]:
#file = 'facebook_combined.txt'
#G = nx.read_edgelist(file, comments='#', delimiter=' ', create_using=nx.Graph(), nodetype=int, data=False)

file = 'wiki-Vote.txt'
G = nx.read_edgelist(file, comments='#', delimiter='\t', create_using=nx.DiGraph(), nodetype=int, data=False)
# G = nx.read_edgelist(file, comments='#', delimiter=' ', create_using=nx.Graph(), nodetype=int, data=False)

# a dictionary by key k, to be jsoned
#data_dump = {}

# non-generator calls
for k in range(1, 10):
	A = list(map(lambda x: x[1], high_degree_nodes(k, G)))        # heuristic 1: HIGHDEG
	res = evaluate(G, A, 0.01, 100, 'IC')
	print('HIGHDEG', len(A), res[0], res[2], A, sep=' ')                       # k, mean, CI95, the seed set
	A = single_discount_high_degree_nodes(k, G)                   # heuristic 2: SDISC
	res = evaluate(G, A, 0.01, 100, 'IC')
	print('SimpleDiscount', len(A), res[0], res[2], A, sep=' ')                       # k, mean, CI95, the seed set
	A,_=general_greedy(k, G, 0.01, 100, 'IC')                       # heuristic 3: GENGREEDY	
	res = evaluate(G, A, 0.01, 100, 'IC')
	print('GENGREEDY', len(A), res[0], res[2], A, sep=' ')                       # k, mean, CI95, the seed set
#	data_dump[len(A)] = (res[0], res[2], A)

#with open('data.txt', 'w') as F:
#	json.dump(data_dump, F)



HIGHDEG 1 14.7 1.6305739357661768 [2565]
SimpleDiscount 1 14.83 1.343438103375068 [2565]
Simple Greedy Detail 0 766 (14.04, 1.3379552064250884)
GENGREEDY 1 14.75 1.546879633326394 [766]
HIGHDEG 2 27.92 1.7369360315221745 [766, 2565]
SimpleDiscount 2 29.77 1.8927664286963672 [2565, 766]
Simple Greedy Detail 0 2565 (15.83, 1.5828877969079174)
Simple Greedy Detail 1 766 (30.24, 2.117003249501521)
GENGREEDY 2 30.25 2.1376358810611316 [2565, 766]
HIGHDEG 3 41.07 2.1700758515775433 [11, 2565, 766]
SimpleDiscount 3 40.94 2.5628557084627297 [2565, 766, 11]
Simple Greedy Detail 0 2565 (14.35, 1.3265522680995274)
Simple Greedy Detail 1 11 (29.28, 2.6507259129529026)
Simple Greedy Detail 2 766 (40.93, 2.491013496872307)
GENGREEDY 3 41.01 2.2382642199704663 [2565, 11, 766]
HIGHDEG 4 53.99 2.721859981409771 [457, 11, 766, 2565]
SimpleDiscount 4 52.91 3.0247690077756353 [2565, 766, 11, 457]
Simple Greedy Detail 0 766 (14.48, 1.6913971010971967)
Simple Greedy Detail 1 2565 (28.87, 1.721155672680423)
